<span style="color:red">Start Position</span>
<br>
<span style="color:blue">Current Position</span>
<br>
<span style="color:green">Target Position</span>
<br>


    BLACK = '\033[30m'
    RED = '\033[31m'
    GREEN = '\033[32m'
    YELLOW = '\033[33m'
    BLUE = '\033[34m'
    MAGENTA = '\033[35m'
    CYAN = '\033[36m'
    WHITE = '\033[37m'
    UNDERLINE = '\033[4m'
    RESET = '\033[0m'

In [189]:
class Cursor():
    map = None
    """
    Pos: Array coordinates 
    State: standing, lying horizontally, and lying vertically
           st, lh and lv
    """
    def __init__(self, pos, state):
        self.pos = pos
        self.state = state
        
    def right(self):
        pass
    
    def left(self):
        pass
    
    def up(self):
        pass
    
    def down(self):
        pass
    
    def is_valid_pos(self):
        pass
    

In [263]:
class Map():
    grid = None
    schema = None
    cursor = None
    def __init__(self, schema:'list[list]', start_pos=None, target_pos=None):
        self.schema = schema
        self.start_pos = start_pos
        self.target_pos = target_pos
        self.init_grid()
            
    def init_cursor(self, current_pos=None, state='st'):
        if current_pos is None:
            current_pos = self.start_pos
        self.cursor = Cursor(current_pos, state)
        self.map = self
    
    def init_grid(self):
        self.rows = len(schema)
        self.cols = 0
        for r in self.schema:
            self.cols = max(self.cols, r[1])
        self.cols+=1
        self.grid = []
        for r in self.schema:
            row = [False]*self.cols
            for i in range(r[0], r[1]+1):
                row[i]=True
            self.grid.append(row)
        
    def plot(self):
        print(f'-- Map Plot {self.rows}X{self.cols}')
        for r in range(self.rows):
            print('')
            # Top
            for c in range(self.cols):
                if self.grid[r][c]:
                    col = '+ '
                elif r>0 and self.grid[r-1][c]:
                    col = '+ '
                else:
                    col = '  '
                print(col, end='')
            print('')
            # Middle|
            for c in range(self.cols):
                if self.grid[r][c]:
                    col = '|'
                else:
                    col = ' '
                empty = True
                if self.cursor is not None:
                    if self.cursor.pos == [r,c]:
                        col+='\033[34mX\033[0m'
                        empty= False
                if empty:
                    if self.start_pos == [r, c]:
                        col+='\033[31mX\033[0m'
                        empty= False
                    elif self.target_pos == [r, c]:
                        col+='\033[32mX\033[0m'
                        empty= False
                if empty:
                    col+=' '
                print(col, end='')
            # End
            if r>=self.rows-1:
                print('')
                for c in range(self.cols):
                    if self.grid[r][c]:
                        col = '+ '
                    else:
                        col = '  '
                    print(col, end='')

In [264]:
schema = [
    (0,9),
    (5,8),
    (3, 7)
]

In [265]:
m = Map(schema, [0,1], [2,6])
m.init_cursor([0, 1])

In [266]:
m.start_pos

[0, 1]

In [267]:
m.grid

[[True, True, True, True, True, True, True, True, True, True],
 [False, False, False, False, False, True, True, True, True, False],
 [False, False, False, True, True, True, True, True, False, False]]

In [268]:
m.plot()

-- Map Plot 3X10

+ + + + + + + + + + 
| |X| | | | | | | | 
+ + + + + + + + + + 
          | | | |   
      + + + + + +   
      | | | |X|     
      + + + + +     